# Merge por coordenadas con DuckDB (usando CP)

- Lee inmuebles con columnas latitud/longitud y cp.
- Lee AGEB desde CSVs y asegura latitud/longitud en DOUBLE.
- Join por cp (estandarizado) y elige el AGEB más cercano dentro del mismo código postal.
- Devuelve df_merge y guarda parquet.

In [1]:
import os
import re
import duckdb
import pandas as pd
from unidecode import unidecode
from pyproj import Transformer
from datetime import datetime

# Parámetros de entrada/salida (dinámicos por fecha)
# "2025-08-31", "2025-09-14", "2025-09-20", "2025-09-27", "2025-10-05", "2025-10-11"
run_date = "2025-10-11"
run_date = datetime.strptime(run_date, "%Y-%m-%d").strftime("%Y-%m-%d")

inmuebles_path = f"../../data/processed/inmuebles24_departamentos_coordenadas_{run_date}.parquet"  # requiere latitud/longitud y cp
ageb_input = "../../data/processed/INEGI/colonia/*.csv"  # CSVs de INEGI (glob) o ruta única
output_path = f"../../data/processed/merged_inmuebles24_departamentos_duckdb_cp_{run_date}.parquet"

# Función para estandarizar texto
def estandarizar_texto(texto):
    if pd.isna(texto):
        return ""
    texto = texto.replace("\n", " ")  # Reemplazar saltos de línea por espacios
    texto = texto.replace(",", "")  # Quitar comas
    return unidecode(texto.strip().lower())

In [2]:
# Conexión DuckDB + extensión spatial
con = duckdb.connect(database=":memory:")
try:
    con.execute("INSTALL spatial;")
except Exception:
    pass
con.execute("LOAD spatial;")

# Cargar AGEB desde CSVs a pandas y transformar coordenadas (INEGI EPSG:6372 -> WGS84 EPSG:4326)
df_ageb = con.execute(f"""
SELECT *
FROM read_csv_auto('{ageb_input}', ignore_errors=True, all_varchar=True)
""").fetchdf()

# Estandarizar postcode en AGEB
if 'postcode' in df_ageb.columns:
    df_ageb['postcode'] = df_ageb['postcode'].astype(str).str.zfill(5)

if {'lon','lat'}.issubset(df_ageb.columns):
    # Convertir a float y transformar con pyproj
    df_ageb[['lon','lat']] = df_ageb[['lon','lat']].astype(float)
    transformer = Transformer.from_crs("epsg:6372", "epsg:4326", always_xy=True)
    lon_wgs, lat_wgs = transformer.transform(df_ageb['lon'].values, df_ageb['lat'].values)
    df_ageb['longitud'] = lon_wgs
    df_ageb['latitud'] = lat_wgs
elif {'longitud','latitud'}.issubset(df_ageb.columns):
    # Ya vienen en WGS84, asegurar tipo float
    df_ageb[['longitud','latitud']] = df_ageb[['longitud','latitud']].astype(float)
else:
    raise ValueError("AGEB necesita columnas lon/lat o longitud/latitud para la transformación.")

# Registrar tabla ageb en DuckDB
con.register("ageb_df", df_ageb)
con.execute("CREATE OR REPLACE TABLE ageb AS SELECT * FROM ageb_df")
con.execute("ALTER TABLE ageb ALTER COLUMN postcode TYPE VARCHAR")

# Cargar inmuebles y estandarizar cp
df_inmuebles = pd.read_parquet(inmuebles_path)
if 'cp' in df_inmuebles.columns:
    df_inmuebles['cp'] = df_inmuebles['cp'].astype(str).str.zfill(5)

con.register("inmuebles_df", df_inmuebles)
con.execute("CREATE OR REPLACE TABLE inmuebles AS SELECT * FROM inmuebles_df")

# Validar columnas de coordenadas en inmuebles
inm_cols = [r[1] for r in con.execute("PRAGMA table_info('inmuebles')").fetchall()]
if not {'latitud','longitud','cp'}.issubset(set(inm_cols)):
    raise ValueError("El parquet de inmuebles debe incluir columnas 'latitud', 'longitud' y 'cp'.")
con.execute("ALTER TABLE inmuebles ALTER COLUMN latitud TYPE DOUBLE")
con.execute("ALTER TABLE inmuebles ALTER COLUMN longitud TYPE DOUBLE")
con.execute("ALTER TABLE inmuebles ALTER COLUMN cp TYPE VARCHAR")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [3]:
# Join por cp: toma el AGEB más cercano dentro del mismo código postal
query = """
WITH inmuebles_validos AS (
    SELECT * 
    FROM inmuebles
    WHERE latitud IS NOT NULL AND longitud IS NOT NULL AND cp IS NOT NULL
)
SELECT 
  i.*, 
  a.*, 
  ST_Distance(ST_Point(i.longitud, i.latitud), ST_Point(a.longitud, a.latitud)) AS distancia
FROM inmuebles_validos i
LEFT JOIN LATERAL (
  SELECT a.*
  FROM ageb a
  WHERE a.postcode = i.cp
  ORDER BY ST_Distance(ST_Point(i.longitud, i.latitud), ST_Point(a.longitud, a.latitud)) ASC
  LIMIT 1
) a ON TRUE
"""

df_merge = con.execute(query).fetchdf()
print("df_merge shape:", df_merge.shape)
df_merge.head(3)

df_merge shape: (4172, 152)


,precio_mxn,lote_m2,recamaras,baños,estacionamiento,es_amueblado,es_penthouse,direccion,colonia,cp,...,lat,municipio_1,address,road,quarter,borough,postcode,longitud_1,latitud_1,distancia
0,13000.0,100,2.0,1.0,0.0,0,0,cerro de la silla 28 campestre churubusco coy...,campestre,01040,...,820010.334701,alvaro_obregon,"{'road': 'Avenida Revolución', 'neighbourhood'...",Avenida Revolución,None,Álvaro Obregón,01040,-99.190509,19.351696,0.057000
1,90000.0,260,2.0,2.0,2.0,1,0,c.monte camerun 145 lomas de chapultepec migu...,hidalgo,01120,...,825407.890601,alvaro_obregon,"{'amenity': 'Colegio Americano', 'road': 'Call...",Calle Tula,None,Álvaro Obregón,01120,-99.206255,19.400896,0.031069
2,63000.0,99,2.0,2.0,1.0,1,0,vazquez de mella polanco miguel hidalgo,hidalgo,01120,...,825638.441185,alvaro_obregon,"{'road': 'Avenida Observatorio', 'neighbourhoo...",Avenida Observatorio,None,Álvaro Obregón,01120,-99.198046,19.402833,0.035237


In [4]:
# Estadísticas de pérdida de datos por falta de coordenadas/cp
total_inmuebles = len(df_inmuebles)
inmuebles_procesados = len(df_merge)
perdidos = total_inmuebles - inmuebles_procesados
porcentaje_perdido = (perdidos / total_inmuebles) * 100 if total_inmuebles > 0 else 0

print(f"Estadísticas para la fecha de ejecución: {run_date}")
print(f"Total de inmuebles inicial: {total_inmuebles}")
print(f"Inmuebles con coordenadas y cp válidos (procesados): {inmuebles_procesados}")
print(f"Inmuebles perdidos: {perdidos} ({porcentaje_perdido:.2f}%)")

Estadísticas para la fecha de ejecución: 2025-10-11
Total de inmuebles inicial: 5975
Inmuebles con coordenadas y cp válidos (procesados): 4172
Inmuebles perdidos: 1803 (30.18%)


In [5]:
# Guardar parquet final
# Guardar parquet final
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df_merge.to_parquet(output_path, index=False)
print("Parquet escrito en:", output_path)

Parquet escrito en: ../../data/processed/merged_inmuebles24_departamentos_duckdb_cp_2025-10-11.parquet


- Estadísticas para la fecha de ejecución: 2025-08-31
- Total de inmuebles inicial: 5858
- Inmuebles con coordenadas y cp válidos (procesados): 4094
- Inmuebles perdidos: 1764 (30.11%)

- Estadísticas para la fecha de ejecución: 2025-09-14
- Total de inmuebles inicial: 6040
- Inmuebles con coordenadas y cp válidos (procesados): 4197
- Inmuebles perdidos: 1843 (30.51%)

- Estadísticas para la fecha de ejecución: 2025-09-20
- Total de inmuebles inicial: 5684
- Inmuebles con coordenadas y cp válidos (procesados): 3982
- Inmuebles perdidos: 1702 (29.94%)

- Estadísticas para la fecha de ejecución: 2025-09-27
- Total de inmuebles inicial: 5986
- Inmuebles con coordenadas y cp válidos (procesados): 4180
- Inmuebles perdidos: 1806 (30.17%)

- Estadísticas para la fecha de ejecución: 2025-10-05
- Total de inmuebles inicial: 5843
- Inmuebles con coordenadas y cp válidos (procesados): 4038
- Inmuebles perdidos: 1805 (30.89%)

- Estadísticas para la fecha de ejecución: 2025-10-11
- Total de inmuebles inicial: 5975
- Inmuebles con coordenadas y cp válidos (procesados): 4172
- Inmuebles perdidos: 1803 (30.18%)